In [2]:
import pandas as pd
from shapely import wkt
import geopandas as gpd
import os
from io import StringIO

In [15]:
# Load the CSV file
df = pd.read_csv('MPS LSOA Level Crime.csv')
# Display the first few rows of the DataFrame
print(df.head())

month_columns = [col for col in df.columns if col.isdigit() and col.startswith("20")]
df['Total'] = df[month_columns].sum(axis=1)
df = df.drop(columns=['Major Category', 'Minor Category','Refreshed Date'])

df.head()

df.to_csv("PhoneTheftData.csv", index=False)


   LSOA Code                  LSOA Name    Borough Major Category  \
0  E01000006  Barking and Dagenham 016A  E09000002          THEFT   
1  E01000007  Barking and Dagenham 015A  E09000002          THEFT   
2  E01000008  Barking and Dagenham 015B  E09000002          THEFT   
3  E01000009  Barking and Dagenham 016B  E09000002          THEFT   
4  E01000011  Barking and Dagenham 016C  E09000002          THEFT   

  Minor Category  202302  202303  202304  202305  202306  ...  202405  202406  \
0    PHONE THEFT       0       1       0       0       0  ...       0       0   
1    PHONE THEFT      10       3       2       5       8  ...      11       9   
2    PHONE THEFT       2       0       0       1       0  ...       0       0   
3    PHONE THEFT       0       1       1       3       3  ...       1       4   
4    PHONE THEFT       0       0       0       0       1  ...       0       0   

   202407  202408  202409  202410  202411  202412  202501  Refreshed Date  
0       0       0     

In [11]:
csv_file = 'PhoneTheftData.csv'
shape_dir = 'LB_shp/'
output_csv = 'PhoneTheftDataWithColor(Geometry).csv'

month_cols = [
    '202302','202303','202304','202305','202306','202307',
    '202308','202309','202310','202311','202312',
    '202401','202402','202403','202404','202405','202406',
    '202407','202408','202409','202410','202411','202412',
    '202501'
]

phoneTheftData = pd.read_csv(csv_file)
phoneTheftData['LSOA_Prefix'] = phoneTheftData['LSOA Name'].str.rsplit(' ', n=1).str[0]
phoneTheftData['LSOA Code'] = phoneTheftData['LSOA Code'].astype(str)

all_data = []

for shp_file in os.listdir(shape_dir):
    if shp_file.endswith('.shp'):
        try:
            shp_path = os.path.join(shape_dir, shp_file)
            gdf = gpd.read_file(shp_path)
            gdf.columns = gdf.columns.str.lower()
            gdf = gdf.to_crs(epsg=4326)
            if {'lsoa21cd', 'lsoa21nm', 'lad22cd', 'lad22nm'}.issubset(gdf.columns):
                gdf['lsoa21cd'] = gdf['lsoa21cd'].astype(str)
                all_data.append(gdf[['lsoa21cd', 'lsoa21nm', 'lad22cd', 'lad22nm', 'geometry']])
        except Exception as e:
            print(f"Error processing {shp_file}: {e}")

all_data_gdf = pd.concat(all_data, ignore_index=True)

processed_data = []

for _, row in phoneTheftData.iterrows():
    lsoa_code = row['LSOA Code']
    match = all_data_gdf[all_data_gdf['lsoa21cd'] == lsoa_code]

    row_data = {
        'LSOA Code': lsoa_code,
        'LSOA Name': row['LSOA Name'],
        'Borough': None,
        'Borough Name': None,
        'geometry': None
    }

    for month in month_cols:
        row_data[month] = row[month] if month in row else 0

    row_data['Total'] = row['Total'] if 'Total' in row else 0

    if not match.empty:
        match_row = match.iloc[0]
        row_data['LSOA Name'] = match_row['lsoa21nm']
        row_data['Borough'] = match_row['lad22cd']
        row_data['Borough Name'] = match_row['lad22nm']
        row_data['geometry'] = match_row['geometry'].simplify(0.00001)

    processed_data.append(row_data)

remaining_lsoa = all_data_gdf[~all_data_gdf['lsoa21cd'].isin(phoneTheftData['LSOA Code'])]

for _, row in remaining_lsoa.iterrows():
    row_data = {
        'LSOA Code': row['lsoa21cd'],
        'LSOA Name': row['lsoa21nm'],
        'Borough': row['lad22cd'],
        'Borough Name': row['lad22nm'],
        'geometry': row['geometry'].simplify(0.00001)
    }

    for month in month_cols:
        row_data[month] = 0

    row_data['Total'] = 0

    processed_data.append(row_data)

final_df = pd.DataFrame(processed_data)
final_df.to_csv(output_csv, index=False)

print(f"Data with geometry saved to: {output_csv}")
print("Geometry column summary:")
print(final_df['geometry'].head())
print("Number of missing geometries:", final_df['geometry'].isnull().sum())

Data with geometry saved to: PhoneTheftDataWithColor(Geometry).csv
Geometry column summary:
0    POLYGON ((0.0898350857932359 51.53982417332992...
1    POLYGON ((0.0776356122208071 51.54280048347972...
2    POLYGON ((0.068891671808709 51.54171902918915,...
3    POLYGON ((0.0837975002438877 51.54203365900032...
4    POLYGON ((0.0839140165805131 51.54292735147401...
Name: geometry, dtype: object
Number of missing geometries: 0


In [28]:
# Load the CSV file
df = pd.read_csv('PhoneTheftDataWithGeometry.csv')

# Define a function to assign colors based on total theft count
def get_color(total):
    if total > 300:
        return '#8B0000'  # Dark Red
    elif total > 200:
        return '#B22222'  # Firebrick
    elif total > 100:
        return '#FF8C00'  # Dark Orange
    elif total > 50:
        return '#FFD700'  # Gold
    elif total > 10:
        return '#90EE90'  # Light Green
    else:
        return '#ADFF2F'  # Green Yellow

# Apply the color function to each row
df['color'] = df['Total'].apply(get_color)

# Save the updated DataFrame to a new CSV file
df.to_csv('PhoneTheftDataWithColor(Geometry).csv', index=False)

print("New file generated: PhoneTheftDataWithColor.csv")

New file generated: PhoneTheftDataWithColor.csv


In [12]:
input_csv = 'PhoneTheftDataWithColor(Geometry).csv'  
output_geojson = 'PhoneTheftDataWithColor(Geometry).geojson' 

df = pd.read_csv(input_csv)

df['geometry'] = df['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry')

gdf.to_file(output_geojson, driver='GeoJSON')
print(f"GeoJSON file successfully created: {output_geojson}")

print(gdf.head())

/opt/anaconda3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


GeoJSON file successfully created: PhoneTheftDataWithColor(Geometry).geojson
   LSOA Code                  LSOA Name    Borough          Borough Name  \
0  E01000006  Barking and Dagenham 016A  E09000002  Barking and Dagenham   
1  E01000007  Barking and Dagenham 015A  E09000002  Barking and Dagenham   
2  E01000008  Barking and Dagenham 015B  E09000002  Barking and Dagenham   
3  E01000009  Barking and Dagenham 016B  E09000002  Barking and Dagenham   
4  E01000011  Barking and Dagenham 016C  E09000002  Barking and Dagenham   

                                            geometry  202302  202303  202304  \
0  POLYGON ((0.08984 51.53982, 0.09227 51.53847, ...       0       1       0   
1  POLYGON ((0.07764 51.5428, 0.07774 51.54279, 0...      10       3       2   
2  POLYGON ((0.06889 51.54172, 0.06905 51.54156, ...       2       0       0   
3  POLYGON ((0.0838 51.54203, 0.08419 51.54183, 0...       0       1       1   
4  POLYGON ((0.08391 51.54293, 0.08475 51.54249, ...       0      

In [13]:
csv_file = 'PhoneTheft_Total_Only.csv'
shape_dir = 'LB_shp/'
output_csv = 'PhoneTheftTotalWithColor(Geometry).csv'

phoneTheftData = pd.read_csv(csv_file)
phoneTheftData['LSOA_Prefix'] = phoneTheftData['LSOA Name'].str.rsplit(' ', n=1).str[0]
phoneTheftData['LSOA Code'] = phoneTheftData['LSOA Code'].astype(str)

all_data = []

for shp_file in os.listdir(shape_dir):
    if shp_file.endswith('.shp'):
        try:
            shp_path = os.path.join(shape_dir, shp_file)
            gdf = gpd.read_file(shp_path)
            gdf.columns = gdf.columns.str.lower()
            gdf = gdf.to_crs(epsg=4326)
            if {'lsoa21cd', 'lsoa21nm', 'lad22cd', 'lad22nm'}.issubset(gdf.columns):
                gdf['lsoa21cd'] = gdf['lsoa21cd'].astype(str)
                all_data.append(gdf[['lsoa21cd', 'lsoa21nm', 'lad22cd', 'lad22nm', 'geometry']])
        except Exception as e:
            print(f"Error processing {shp_file}: {e}")

all_data_gdf = pd.concat(all_data, ignore_index=True)

processed_data = []

for _, row in phoneTheftData.iterrows():
    lsoa_code = row['LSOA Code']
    match = all_data_gdf[all_data_gdf['lsoa21cd'] == lsoa_code]

    row_data = {
        'LSOA Code': lsoa_code,
        'LSOA Name': row['LSOA Name'],
        'Borough': None,
        'Borough Name': None,
        'geometry': None
    }

    row_data['Total'] = row['Total'] if 'Total' in row else 0

    if not match.empty:
        match_row = match.iloc[0]
        row_data['LSOA Name'] = match_row['lsoa21nm']
        row_data['Borough'] = match_row['lad22cd']
        row_data['Borough Name'] = match_row['lad22nm']
        row_data['geometry'] = match_row['geometry'].simplify(0.00001)

    processed_data.append(row_data)

remaining_lsoa = all_data_gdf[~all_data_gdf['lsoa21cd'].isin(phoneTheftData['LSOA Code'])]

for _, row in remaining_lsoa.iterrows():
    row_data = {
        'LSOA Code': row['lsoa21cd'],
        'LSOA Name': row['lsoa21nm'],
        'Borough': row['lad22cd'],
        'Borough Name': row['lad22nm'],
        'geometry': row['geometry'].simplify(0.00001)
    }

    row_data['Total'] = 0

    processed_data.append(row_data)

final_df = pd.DataFrame(processed_data)
final_df.to_csv(output_csv, index=False)

print(f"Data with geometry saved to: {output_csv}")
print("Geometry column summary:")
print(final_df['geometry'].head())
print("Number of missing geometries:", final_df['geometry'].isnull().sum())

Data with geometry saved to: PhoneTheftTotalWithColor(Geometry).csv
Geometry column summary:
0    POLYGON ((0.0898350857932359 51.53982417332992...
1    POLYGON ((0.0776356122208071 51.54280048347972...
2    POLYGON ((0.068891671808709 51.54171902918915,...
3    POLYGON ((0.0837975002438877 51.54203365900032...
4    POLYGON ((0.0839140165805131 51.54292735147401...
Name: geometry, dtype: object
Number of missing geometries: 0


In [16]:
csv_file = 'PhoneTheftTotalWithColor(Geometry).csv'
output_csv = 'PhoneTheft_Total_Only.csv'
df = pd.read_csv(csv_file)
df = df[['LSOA Code', 'Total']]
df.to_csv(output_csv, index=False)